# Grouping and Aggregating - Spend Type and Order Frequency

1. Import Libraries and Dataset
2. Mean Orders by Department
3. Create a variable for customer spending type
4. Create a variable for customer order frequency
5. Export data with new variables

# 01. Import Libraries and Dataset

In [2]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [3]:
path = r'C:\Users\lizan\Desktop\Data Analytics\4.0\31-05-2022 Instacart Basket Analysis'

In [4]:
ords_prods_merge = pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','orders_products_busy_loyalty.pkl'))

In [4]:
ords_prods_merge.shape

(32404859, 21)

In [5]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,_merge,price_range_loc,Busiest_Day,Busiest_Days,Busiest_Hours,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,...,77,7,9.0,both,Mid Range Produt,Regularly Busy Day,Regular Days,Average Orders,10,New Customer
1,2398795,1,2,3,7,15.0,False,196,1,1,...,77,7,9.0,both,Mid Range Produt,Regularly Busy Day,Slowest Days,Average Orders,10,New Customer
2,473747,1,3,3,12,21.0,False,196,1,1,...,77,7,9.0,both,Mid Range Produt,Regularly Busy Day,Slowest Days,Most Orders,10,New Customer
3,2254736,1,4,4,7,29.0,False,196,1,1,...,77,7,9.0,both,Mid Range Produt,Least Busy Day,Slowest Days,Average Orders,10,New Customer
4,431534,1,5,4,15,28.0,False,196,1,1,...,77,7,9.0,both,Mid Range Produt,Least Busy Day,Slowest Days,Most Orders,10,New Customer


With more data, the mean values charged compared to the subset of on million entries.  A majority decreased, but a couple increased.

# 03. Customer Type Spending

In [10]:
# Basic stats for customer spending per product by customer type
ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal Customer    10.386336  1.0  99999.0
New Customer      13.294670  1.0  99999.0
Regular Customer  12.495717  1.0  99999.0

On average, Loyal Customers spend less perproduct versus a New or Regular customer.

Criteria for spending flag
If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”

If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [11]:
ords_prods_merge.groupby('user_id').agg({'prices':['mean']})

,prices
,mean
user_id,
1,6.367797
2,7.515897
3,8.197727
4,8.205556
5,9.189189
...,...
206205,8.909375
206206,7.646667


In [5]:
# Add column for the average spend/product for each user
ords_prods_merge['avg_spend'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [6]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,department_id,prices,_merge,price_range_loc,Busiest_Day,Busiest_Days,Busiest_Hours,max_order,loyalty_flag,avg_spend
0,2539329,1,1,2,8,NaN,True,196,1,0,...,7,9.0,both,Mid Range Produt,Regularly Busy Day,Regular Days,Average Orders,10,New Customer,6.367797
1,2398795,1,2,3,7,15.0,False,196,1,1,...,7,9.0,both,Mid Range Produt,Regularly Busy Day,Slowest Days,Average Orders,10,New Customer,6.367797
2,473747,1,3,3,12,21.0,False,196,1,1,...,7,9.0,both,Mid Range Produt,Regularly Busy Day,Slowest Days,Most Orders,10,New Customer,6.367797
3,2254736,1,4,4,7,29.0,False,196,1,1,...,7,9.0,both,Mid Range Produt,Least Busy Day,Slowest Days,Average Orders,10,New Customer,6.367797
4,431534,1,5,4,15,28.0,False,196,1,1,...,7,9.0,both,Mid Range Produt,Least Busy Day,Slowest Days,Most Orders,10,New Customer,6.367797


In [7]:
# Derive a column for spend type
ords_prods_merge.loc[ords_prods_merge['avg_spend'] < 10, 'spend_type'] = 'Low Spender'

In [8]:
ords_prods_merge.loc[ords_prods_merge['avg_spend'] >= 10, 'spend_type'] = 'High Spender'

In [9]:
ords_prods_merge['spend_type'].value_counts(dropna=False)

Low Spender     31770614
High Spender      634245
Name: spend_type, dtype: int64

In [10]:
# Check
ords_prods_merge[['user_id', 'spend_type']].head(80)
                                            

,user_id,spend_type
0,1,Low Spender
1,1,Low Spender
2,1,Low Spender
3,1,Low Spender
4,1,Low Spender
...,...,...
75,120,Low Spender
76,120,Low Spender
77,185,Low Spender
78,195,Low Spender


# 04. Customer Order Frequency

In [11]:
# Add column to customer median order frequency
ords_prods_merge['median_order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [12]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,_merge,price_range_loc,Busiest_Day,Busiest_Days,Busiest_Hours,max_order,loyalty_flag,avg_spend,spend_type,median_order_frequency
0,2539329,1,1,2,8,NaN,True,196,1,0,...,both,Mid Range Produt,Regularly Busy Day,Regular Days,Average Orders,10,New Customer,6.367797,Low Spender,20.5
1,2398795,1,2,3,7,15.0,False,196,1,1,...,both,Mid Range Produt,Regularly Busy Day,Slowest Days,Average Orders,10,New Customer,6.367797,Low Spender,20.5
2,473747,1,3,3,12,21.0,False,196,1,1,...,both,Mid Range Produt,Regularly Busy Day,Slowest Days,Most Orders,10,New Customer,6.367797,Low Spender,20.5
3,2254736,1,4,4,7,29.0,False,196,1,1,...,both,Mid Range Produt,Least Busy Day,Slowest Days,Average Orders,10,New Customer,6.367797,Low Spender,20.5
4,431534,1,5,4,15,28.0,False,196,1,1,...,both,Mid Range Produt,Least Busy Day,Slowest Days,Most Orders,10,New Customer,6.367797,Low Spender,20.5


Derive Column for Frequency on the following criteria: 
If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”

If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”

If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [13]:
ords_prods_merge.loc[ords_prods_merge['median_order_frequency'] > 20, 'frequency_flag'] = 'Non-Frequent Customer'

In [14]:
ords_prods_merge.loc[(ords_prods_merge['median_order_frequency'] > 10) & (ords_prods_merge['median_order_frequency'] <= 20), 'frequency_flag'] = 'Regular Customer'

In [15]:
ords_prods_merge.loc[ords_prods_merge['median_order_frequency'] <= 10, 'frequency_flag'] = 'Frequent Customer'

In [16]:
ords_prods_merge['frequency_flag'].value_counts(dropna=False)

Frequent Customer        21559853
Regular Customer          7208564
Non-Frequent Customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

In [17]:
# Check
ords_prods_merge[['user_id', 'frequency_flag', 'median_order_frequency']].head(60)

,user_id,frequency_flag,median_order_frequency
0,1,Non-Frequent Customer,20.5
1,1,Non-Frequent Customer,20.5
2,1,Non-Frequent Customer,20.5
3,1,Non-Frequent Customer,20.5
4,1,Non-Frequent Customer,20.5
5,1,Non-Frequent Customer,20.5
6,1,Non-Frequent Customer,20.5
7,1,Non-Frequent Customer,20.5
8,1,Non-Frequent Customer,20.5
9,1,Non-Frequent Customer,20.5


In [18]:
ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_time', 'days_since_prior_order', 'new_customer', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', '_merge', 'price_range_loc', 'Busiest_Day',
       'Busiest_Days', 'Busiest_Hours', 'max_order', 'loyalty_flag',
       'avg_spend', 'spend_type', 'median_order_frequency', 'frequency_flag'],
      dtype='object')

In [19]:
ords_prods_merge.loc[ords_prods_merge['prices'] > 100, 'prices'] = np.nan

#  05. Export

In [20]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_spend_freq2.pkl'))